## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [10]:
data = range(1, 1001)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [11]:
import pyspark
sc = pyspark.SparkContext('local[*]')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-9ae820830206>:2 

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [12]:
rdd = sc.parallelize(data, numSlices = 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [13]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [14]:
# count
rdd.count()

1000

In [15]:
# first
rdd.first()

1

In [17]:
# take
rdd.take(12)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [18]:
# top
rdd.top(3)

[1000, 999, 998]

In [19]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [20]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([2.14086143e-01, 8.40946939e-01, 3.19542713e-01, 3.61162430e+00,
       2.10754697e+00, 5.61524133e+00, 2.13322176e+00, 2.73054217e+00,
       4.63703075e-01, 7.54003229e+00, 5.28842489e+00, 1.09538038e+01,
       4.73811468e+00, 1.06678088e+01, 1.62209892e+00, 8.82091860e+00,
       1.08915783e+01, 2.87121774e+00, 8.34951610e+00, 2.86858187e+00,
       1.94520629e+01, 2.40785358e+00, 1.65913313e+01, 2.31730771e+01,
       5.32918368e+00, 2.12586069e+01, 1.18550725e+01, 8.75217134e+00,
       1.89036555e+01, 2.34500827e+01, 2.86624954e+01, 3.27039658e+00,
       2.26061492e+01, 3.31956948e+00, 1.94114371e+01, 4.78428484e+00,
       2.46905115e+01, 6.34156371e+00, 1.27910176e+01, 2.60535489e+01,
       3.14518790e+01, 2.15226677e+01, 2.07296752e+01, 1.08189336e+01,
       3.61074314e+01, 1.35497705e+00, 3.76857595e+01, 1.26426322e+01,
       5.97094472e+00, 1.44969634e+01, 2.23260829e+01, 3.77159938e+01,
       1.91346490e+01, 2.42963747e+01, 4.95145381e+00, 1.82612304e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [22]:
price_items = sc.parallelize(sales_figures, numSlices = 10)
price_items.getNumPartitions()

10

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [23]:
def sales_tax(num):
    return num * .92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [24]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.19695925195147393,
 0.773671184149044,
 0.29397929606931134,
 3.3226943557022004,
 1.9389432135548927,
 5.166022027295084,
 1.9625640187672981,
 2.5120987938272608,
 0.4266068285564703,
 6.93682970493783]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [25]:
discounted = price_items.map(lambda x: x*0.9)

In [26]:
discounted.take(10)

[0.19267752908296362,
 0.7568522453631953,
 0.287588441806935,
 3.250461869708674,
 1.8967922741297862,
 5.053717200614757,
 1.9198995835767048,
 2.45748795048319,
 0.4173327670661123,
 6.786029059178312]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [27]:
price_items.map(sales_tax).map(lambda x: x*.9).top(15)

[804.8687787231421,
 793.2248083526887,
 787.066776290177,
 770.6961484800863,
 755.9115698047757,
 735.5575123285544,
 721.2316152005254,
 719.5731343172685,
 712.5120329709204,
 707.4231697176879,
 707.0422200687972,
 704.205492732243,
 697.1602429240875,
 694.5777341914907,
 681.347775076683]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [28]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [31]:
mapped = price_items.map(lambda x: (x, x*0.92 * .9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.21408614342551513, 0.17726332675632653), (0.8409469392924391, 0.6963040657341396), (0.31954271311881666, 0.26458136646238023), (3.6116242996763046, 2.9904249201319804), (2.107546971255318, 1.7450488921994034), (5.615241334016396, 4.649419824565576), (2.133221759529672, 1.7663076168905683), (2.730542167203544, 2.2608889144445348), (0.4637030745179025, 0.3839461457008233), (7.540032287975902, 6.243146734444047)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [32]:
flat_mapped = price_items.flatMap(lambda x: (x, x* 0.92 * 0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.21408614342551513, 0.17726332675632653, 0.8409469392924391, 0.6963040657341396, 0.31954271311881666, 0.26458136646238023, 3.6116242996763046, 2.9904249201319804, 2.107546971255318, 1.7450488921994034]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [33]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)
selected_items.count()
# calculate total remaining in inventory 


301

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [34]:
selected_items.reduce(lambda x,y: x + y)

146632.31681362694

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [35]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(7, 305.58161544239744),
 (22, 321.6053742443906),
 (31, 326.04701262056255),
 (35, 325.2264314636324),
 (7, 323.2852925378184),
 (28, 342.1726334422936),
 (45, 304.5385361882138)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [37]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x+ y)

In [39]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending = False).collect()

[(20, 6454.393253786699),
 (43, 5229.278629638038),
 (26, 5033.577578124587),
 (47, 4718.306216647468),
 (1, 4700.916408916497),
 (7, 4324.474098400234),
 (32, 4129.275292390089),
 (45, 4109.02256150013),
 (16, 4105.78224078877),
 (29, 4060.59406817193),
 (50, 3963.4667153079827),
 (36, 3882.924910299819),
 (4, 3854.1290084377993),
 (44, 3836.4873523093374),
 (19, 3702.968820700213),
 (13, 3653.0584966964934),
 (24, 3436.5056836248036),
 (5, 3425.6328838253407),
 (40, 3181.5754050655933),
 (41, 3145.741256722081),
 (39, 3014.7536558657957),
 (2, 2904.3802723117615),
 (42, 2898.621231344952),
 (31, 2867.9536954489163),
 (49, 2859.9967178742477),
 (30, 2620.500090499998),
 (3, 2522.9476605840064),
 (8, 2485.046561532286),
 (9, 2404.1519780837225),
 (46, 2230.3643790066735),
 (38, 2209.7517192105283),
 (10, 2209.64064169802),
 (27, 2197.737838956058),
 (33, 2188.0194980597744),
 (25, 2107.0430607049943),
 (18, 2089.971048372453),
 (48, 2079.7908948498907),
 (35, 2034.0660108475445),
 (6, 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [40]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key = lambda kv: kv[1], reverse = True)

[(40, 12),
 (9, 12),
 (17, 10),
 (18, 9),
 (45, 9),
 (39, 9),
 (50, 9),
 (31, 9),
 (15, 8),
 (3, 7),
 (42, 7),
 (2, 7),
 (33, 7),
 (49, 7),
 (13, 7),
 (34, 7),
 (23, 7),
 (16, 6),
 (6, 6),
 (46, 6),
 (8, 6),
 (26, 6),
 (28, 6),
 (5, 6),
 (20, 6),
 (35, 6),
 (43, 6),
 (36, 6),
 (41, 5),
 (32, 5),
 (14, 5),
 (21, 5),
 (30, 5),
 (4, 5),
 (12, 5),
 (22, 5),
 (48, 5),
 (47, 5),
 (29, 4),
 (44, 4),
 (24, 4),
 (37, 4),
 (7, 4),
 (11, 4),
 (1, 4),
 (38, 4),
 (10, 3),
 (19, 3),
 (25, 3),
 (27, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
